In [1]:
import json, math
import pandas as pd

In [2]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import pandas as pd
import json
import numpy as np
from collections import Counter
import ast

def Hindex(citations):
    n = len(citations)
    citations.sort(reverse=True)  # 将引用量列表按降序排序

    h_index = 0
    for i in range(n):
        if citations[i] >= i + 1:
            h_index = i + 1
        else:
            break
    return h_index

def Gindex(indexList):
#     indexList = [int(num) for num in indexList]
    #构建引用次数与文章篇数的映射
    HCounter = Counter(indexList)
    #逆序字典，让引用次数高的引用次数排在前面
    ReversedCounter = sorted(HCounter.items(), reverse = True)
    #分别生成 引用次数列表CounterKeys 和 该引用次数的文章列表CounterValues
    CounterKeys = [i[0] for i in ReversedCounter ]
    CounterValues = [i[1] for i in ReversedCounter]
    #CounterKeys，CounterValues根据索引值一一对应，遍历索引值
    for index in range(0,len(CounterValues)):
        #sum(CounterValues[0:index+1])为大于等于某个索引值——CounterKeys[index]的所有的文章总和
        if sum([CounterKeys[i]*CounterValues[i] for i in range(index)]) <= math.pow(sum(CounterValues[0:index+1]),2):
            break
    return CounterKeys[index]
# 判断字符重合
def compare_letters(str1, str2):
    str1 = str(str1)
    str2 = str(str2)
    # 去除字符串中的非字母字符，并转换为小写字母
    letters1 = ''.join(filter(str.isalpha, str1.lower()))
    letters2 = ''.join(filter(str.isalpha, str2.lower()))

    # 判断两个字符串中的字母是否相同
    return sorted(letters1) == sorted(letters2)
# 判断字符重合度
def check_letter_overlap(str1, str2):
    str1 = str(str1)
    str2 = str(str2)
    # 将字符串中的非英文字母字符去除，转换为小写
    str1 = ''.join(filter(str.isalpha, str1.lower()))
    str2 = ''.join(filter(str.isalpha, str2.lower()))
    
    # 统计两个字符串中的英文字母个数
    count1 = len(str1)
    count2 = len(str2)
    
    # 统计两个字符串中重叠的英文字母个数
    overlap_count = sum([1 for char in str1 if char in str2])
    
    # 计算重叠度
    overlap_percentage = (overlap_count / max(count1, count2)) * 100
    
    # 判断重叠度是否超过90%
    return overlap_percentage
# 返回索引
def sort_with_indices(numbers):
    # Enumerate the numbers and store them as tuples with their indices
    enumerated_numbers = list(enumerate(numbers))
    
    # Sort the enumerated numbers in descending order based on the values
    sorted_numbers = sorted(enumerated_numbers, key=lambda x: x[1], reverse=True)
    
    # Extract the indices from the sorted list and return them
    indices = [x[0] for x in sorted_numbers]
    return indices

In [4]:
df = pd.read_excel('Total Articles of 287 ACM Fellows.xlsx')
df.head()

,Fellow,Title,Year,Authorlist,Cited,Contribution,Position,Persons,Cited_weight
0,"Markl, Volker",query processing on heterogeneous cpugpu systems,2023,"[Viktor Rosenfeld, Sebastian Breß, Volker Markl]",0.0,0.181818,3,3,0.000000
1,"Markl, Volker",the seattle report on database research,2022,"[Daniel Abadi, Anastasia Ailamaki, David G. An...",24.0,0.012229,20,33,0.293485
2,"Balazinska, Magdalena",the seattle report on database research,2022,"[Daniel Abadi, Anastasia Ailamaki, David G. An...",24.0,0.048914,5,33,1.173939
3,"Abadi, Daniel J",the seattle report on database research,2020,"[Daniel Abadi, Anastasia Ailamaki, David Ander...",24.0,0.244571,1,33,5.869695
4,"Markl, Volker",handling iterations in distributed dataflow sy...,2022,"[Gábor E. Gévay, Juan Soto 0001, Volker Markl]",0.0,0.181818,3,3,0.000000


In [17]:
# 从JSON文件中读取数据
with open('D:\投稿\学者成长规律\学者影响力-生产力曲线.json', 'r') as f:
    data = json.load(f)

In [26]:
list1 = []
list111 = [item for item in data.keys()]
for item in df['Fellow'].unique():
    if item in list111:
        list11 = df[df['Fellow']==item]['Cited_weight'].tolist()
        list1.append([item,data[item]['career'],data[item]['total_article'],
                      data[item]['total_cite'],data[item]['per_cite'],Hindex(list11),
                     math.pow(math.pow(data[item]["total_cite"],2)/data[item]["total_article"],1/3)])

In [28]:
# 构建DataFrame
columns1 = ['Name','Age','Productivity','Citation','Per_Citation','Hindex','Pindex']
df = pd.DataFrame(list1, columns=columns1)
df.head()

,Name,Age,Productivity,Citation,Per_Citation,Hindex,Pindex
0,"Markl, Volker",26,18.955984,787.275161,41.531749,11,31.976843
1,"Balazinska, Magdalena",25,18.659556,2998.314493,160.685200,19,78.394276
2,"Abadi, Daniel J",20,12.744294,3578.358613,280.781233,11,100.157616
3,"Madden, Samuel",21,14.421830,15446.768451,1071.068530,28,254.811074
4,"Tao, Yufei",21,14.421830,16677.347173,1156.396030,41,268.170615


In [30]:
df.to_excel('学者大致统计信息.xlsx',index=False)